In [11]:
from FinMind.data import DataLoader
from dotenv import load_dotenv
import os

load_dotenv()
token = os.getenv("API_TOKEN")

api = DataLoader()
api.login_by_token(api_token=token)

In [12]:
start_date = "1994-10-02"
days_ago = 20
stock_id = "2385"

candle_low = 0.03
volume_x = 3

tp = 0.05
sl = -0.03

In [23]:
taiwan_stocks = api.taiwan_stock_info()

# Extract stock ids
stock_ids = taiwan_stocks['stock_id'].tolist()

print(f"Total stocks fetched: {len(stock_ids)}")

2024-03-24 23:38:24.070 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockInfo, data_id: 


Total stocks fetched: 3238


In [24]:
stock_ids

['0050',
 '0051',
 '0052',
 '0053',
 '0054',
 '0055',
 '0056',
 '0057',
 '0058',
 '0059',
 '0060',
 '0061',
 '006201',
 '006203',
 '006204',
 '006205',
 '006206',
 '006207',
 '006208',
 '00625K',
 '00631L',
 '00632R',
 '00633L',
 '00634R',
 '00635U',
 '00636',
 '00636K',
 '00637L',
 '00638R',
 '00639',
 '00640L',
 '00641R',
 '00642U',
 '00643',
 '00643K',
 '00645',
 '00646',
 '00647L',
 '00648R',
 '00649',
 '00650L',
 '00651R',
 '00652',
 '00653L',
 '00654R',
 '00655L',
 '00656R',
 '00657',
 '00657K',
 '00658L',
 '00659R',
 '00660',
 '00661',
 '00662',
 '00663L',
 '00664R',
 '00665L',
 '00666R',
 '00667',
 '00668',
 '00668K',
 '00669R',
 '00670L',
 '00671R',
 '00672L',
 '00673R',
 '00674R',
 '00675L',
 '00676R',
 '00677U',
 '00678',
 '00679B',
 '00680L',
 '00681R',
 '00682U',
 '00683L',
 '00684R',
 '00685L',
 '00686R',
 '00687B',
 '00688L',
 '00689R',
 '00690',
 '00691R',
 '00692',
 '00693U',
 '00694B',
 '00695B',
 '00696B',
 '00697B',
 '00698L',
 '00699R',
 '00700',
 '00701',
 '00702'

In [13]:
import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [14]:
# Get the current time as a datetime object
current_datetime = datetime.now()

# Calculate the datetime for 30 days ago
n_days_ago_datetime = current_datetime - timedelta(days=days_ago)

# Format both dates as 'YYYY-MM-DD'
current_date_str = current_datetime.strftime('%Y-%m-%d')
n_days_ago_date_str = n_days_ago_datetime.strftime('%Y-%m-%d')

print(f"Current Date: {current_date_str}")
print(f"Date {days_ago} Days Ago: {n_days_ago_date_str}")

Current Date: 2024-03-24
Date 20 Days Ago: 2024-03-04


In [15]:
df = api.taiwan_stock_daily_adj(
    stock_id = stock_id,
    start_date = start_date,
    end_date = current_date_str,
)

2024-03-24 23:22:47.131 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPriceAdj, data_id: 2385


In [16]:
df

,date,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover
0,1999-01-06,2385,3216091,147205926,6.76,6.98,6.76,6.82,-0.44,1126
1,1999-01-07,2385,2645941,116716265,6.82,6.83,6.34,6.83,0.01,1006
2,1999-01-08,2385,2096589,96009854,6.61,6.88,6.61,6.85,0.02,770
3,1999-01-11,2385,1329400,59643244,6.83,6.83,6.47,6.70,-0.15,484
4,1999-01-12,2385,1799002,78059456,6.53,6.58,6.31,6.31,-0.39,429
...,...,...,...,...,...,...,...,...,...,...
6084,2024-03-18,2385,13272886,3280932502,240.00,253.00,237.50,253.00,12.50,11707
6085,2024-03-19,2385,19915886,5150614669,254.50,266.50,249.00,258.50,5.50,23430
6086,2024-03-20,2385,14193561,3707496110,260.00,267.00,255.00,258.50,0.00,13250
6087,2024-03-21,2385,18012670,4489194137,262.50,262.50,244.00,247.00,-11.50,15836


In [20]:
def trades:
    # condition 1
    c1_green = ((df['open'] - df['min']) / df['open']) >= candle_low
    c1_red = ((df['close'] - df['min']) / df['open']) >= candle_low
    c1 = ((df['close'] > df['open']) & c1_green) | ((df['open'] > df['close']) & c1_red)
    
    # condition 2
    rolling_avg_volume = df['Trading_Volume'].rolling(window=days_ago).mean()
    c2 = df['Trading_Volume'] >= rolling_avg_volume * volume_x
    
    # Combine conditions
    conditions_met = c1 & c2
    
    conditions_met_shifted = conditions_met.shift(1, fill_value=False)
    
    df['trade_entry'] = conditions_met_shifted
    df['exit_signal'] = False
    df['next_day_open'] = df['open'].shift(-1)
    
    # Iterate through DataFrame to simulate trade entries and exits
    in_trade = False
    entry_price = None
    
    for i in range(len(df)):
        if not in_trade:
            # Check if we should enter a trade
            if df.iloc[i]['trade_entry']:
                in_trade = True
                entry_price = df.iloc[i]['open']
                df.at[i, 'entry_price'] = entry_price
        else:
            # Calculate price changes since entry
            max_pct_change = (df.iloc[i]['max'] - entry_price) / entry_price
            min_pct_change = (df.iloc[i]['min'] - entry_price) / entry_price
            
            exit_based_on_close = (df.iloc[i]['close'] - entry_price) / entry_price
            
            # Check exit criteria: max reaching 5% or min reaching -3%
            if max_pct_change >= tp or min_pct_change <= sl:
                df.at[i, 'exit_signal'] = True
                df.at[i, 'exit_price'] = entry_price * (1 + tp if max_pct_change >= tp else 1 + sl)
                in_trade = False
                entry_price = None
    
    # Filter to find rows where a trade was exited
    trades_exited = df[df['exit_signal'] | df['trade_entry']]
    
    print(trades_exited[['open', 'max', 'min', 'close', 'entry_price', 'exit_price', 'trade_entry', 'exit_signal']])

      open    max   min  close  entry_price  exit_price  trade_entry  \
52    5.66   5.79  5.57   5.76         5.66         NaN         True   
54    5.32   5.58  5.27   5.48          NaN      5.4902        False   
123   4.65   4.90  4.65   4.90         4.65         NaN         True   
126   3.94   3.99  3.70   3.73          NaN      4.5105        False   
675   3.94   3.94  3.79   3.86         3.94         NaN         True   
676   3.83   3.86  3.71   3.77          NaN      3.8218        False   
800   9.80  10.32  9.80  10.17         9.80         NaN         True   
801  10.32  10.32  9.50   9.80          NaN     10.2900        False   

     exit_signal  
52         False  
54          True  
123        False  
126         True  
675        False  
676         True  
800        False  
801         True  


In [21]:
# df.to_csv("data.csv")

In [22]:
# Initialize variables for calculating wins and total profits
total_trades = 0
wins = 0
total_profit = 0.0

# Iterate through DataFrame to calculate wins and total profits
for i in df.index[df['exit_signal']]:
    # Look back to find the first 'trade_entry' before this 'exit_signal'
    entry_index = df.loc[:i][df['trade_entry']].last_valid_index()
    if pd.notnull(entry_index):
        entry_price = df.loc[entry_index, 'entry_price']  # Assume 'entry_price' was filled at trade entry
        exit_price = df.loc[i, 'exit_price']
        profit = exit_price - entry_price
        total_profit += profit
        total_trades += 1
        if profit > 0:
            wins += 1

# Avoid division by zero
win_rate = (wins / total_trades) * 100 if total_trades > 0 else 0

print(f"Win Rate: {win_rate:.2f}%")
print(f"Total Profit: {total_profit}")

Win Rate: 25.00%
Total Profit: 0.062499999999999556


/tmp/ipykernel_15024/799643208.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  entry_index = df.loc[:i][df['trade_entry']].last_valid_index()
/tmp/ipykernel_15024/799643208.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  entry_index = df.loc[:i][df['trade_entry']].last_valid_index()
/tmp/ipykernel_15024/799643208.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  entry_index = df.loc[:i][df['trade_entry']].last_valid_index()
/tmp/ipykernel_15024/799643208.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  entry_index = df.loc[:i][df['trade_entry']].last_valid_index()


In [ ]:
for index, row in stock_ids.iterrows():
    stock_id = row['stock_id']  # Assuming 'stock_id' is the correct column name
    print(f"Fetching data for stock ID: {stock_id}")
    
    # Fetch daily adjusted data for the stock
    df = api.taiwan_stock_daily_adj(
        stock_id=stock_id,
        start_date=start_date,
        end_date=current_date_str,
    )